In [1]:
import numpy as np
import polars as pl
import polars_pbv as pl_pbv
import hvplot
import holoviews as hv
import panel as pn

In [3]:
pn.__version__

'1.4.4'

In [4]:
hv.__version__

'1.18.3'

In [5]:
n = 1000
w = 120
b = 20
df = pl.DataFrame({
    "index": np.arange(0, n),
    "price": np.random.normal(size=n),
    "volume": np.random.randint(0, 1000, n),
}).with_columns(
    pl.col("price").cum_sum()
).with_columns(
    pl_pbv.pbv_pct("price", "volume", w, b, center=False).alias("pbv")
)

In [6]:
df

index,price,volume,pbv
i64,f64,i64,struct[2]
0,0.694999,979,"{null,null}"
1,1.65609,968,"{null,null}"
2,2.348936,922,"{null,null}"
3,1.011861,796,"{null,null}"
4,2.256975,0,"{null,null}"
…,…,…,…
995,35.706383,735,"{[29.585915, 30.208259, … 41.410433],[0.024675, 0.014959, … 0.038207]}"
996,35.475052,45,"{[29.585915, 30.208259, … 41.410433],[0.02496, 0.015132, … 0.038648]}"
997,34.708433,629,"{[29.585915, 30.208259, … 41.410433],[0.024923, 0.015109, … 0.03859]}"


In [7]:
df_pbv = df.tail(1).select(
    pl.col("pbv").struct.field("price").list.explode(),
    pl.col("pbv").struct.field("volume").list.explode() * n,
)
pbv_data = df_pbv.to_dicts()
step = pbv_data[1]["price"] - pbv_data[0]["price"]
pbv_v_max = df_pbv["volume"].max()

In [16]:
((df.tail(w).plot.line(x="index", y="price", height=400) * 
  hv.Rectangles([[n-w, r["price"], (n - w) + r["volume"], r["price"] + step] for r in pbv_data], ).opts(alpha=0.5)
 ) +  df.tail(w).plot.step(x="index", y="volume", height=200)).cols(1)

:Layout
   .Overlay.I :Overlay
      .Curve.I      :Curve   [index]   (price)
      .Rectangles.I :Rectangles   [x0,y0,x1,y1]
   .Curve.I   :Curve   [index]   (volume)

In [9]:
w

120

In [15]:
i = 200
df_pbv = df.slice(i+w, 1).select(
    pl.col("pbv").struct.field("price").list.explode(),
    pl.col("pbv").struct.field("volume").list.explode() * n,
)
pbv_data = df_pbv.to_dicts()
step = pbv_data[1]["price"] - pbv_data[0]["price"]
pbv_v_max = df_pbv["volume"].max()
((df.slice(i, w).plot.line(x="index", y="price", height=400) * 
  hv.Rectangles([[i, r["price"], i + r["volume"], r["price"] + step] for r in pbv_data], ).opts(alpha=0.5)
 ) + df.slice(i, w).plot.step(x="index", y="volume", height=200)).cols(1)

:Layout
   .Overlay.I :Overlay
      .Curve.I      :Curve   [index]   (price)
      .Rectangles.I :Rectangles   [x0,y0,x1,y1]
   .Curve.I   :Curve   [index]   (volume)

In [17]:
def plot_pbv(i):
    print(f"use {i}")
    df_pbv = df.slice(i+w, 1).select(
        pl.col("pbv").struct.field("price").list.explode(),
        pl.col("pbv").struct.field("volume").list.explode() * n,
    )
    pbv_data = df_pbv.to_dicts()
    step = pbv_data[1]["price"] - pbv_data[0]["price"]
    pbv_v_max = df_pbv["volume"].max()
    return ((df.slice(i, w).plot.line(x="index", y="price", height=400) * 
      hv.Rectangles([[i, r["price"], i + r["volume"], r["price"] + step] for r in pbv_data], ).opts(alpha=0.5)
     ) + df.slice(i, w).plot.step(x="index", y="volume", height=200)).cols(1)

In [18]:
slider = pn.widgets.IntSlider(name='time', start=w, end=n, step=10)

In [19]:
pbv_plot_interactive = hvplot.bind(plot_pbv, i=slider)

In [20]:
dashboard = pn.Column(slider, pbv_plot_interactive)
dashboard.servable()
dashboard

use 120


Column
    [0] IntSlider(end=1000, name='time', start=120, step=10, value=120)
    [1] ParamFunction(function, _pane=HoloViews, defer_load=False)

In [24]:
dashboard.save("dashboard.html", embed=True, max_opts=100)

  0%|                                                                                                                          | 0/89 [00:00<?, ?it/s]use 1000
use 990
use 980
use 970
use 960
use 950
use 940
use 930
use 920
use 910
use 900
use 890
use 880
use 870
 16%|█████████████████▊                                                                                               | 14/89 [00:00<00:02, 33.73it/s]use 860
use 850
use 840
use 830
 20%|██████████████████████▊                                                                                          | 18/89 [00:02<00:10,  6.92it/s]use 820
use 810
 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████▋ | 88/89 [00:30<00:00,  2.55it/s]use 120
